# Preparing a proposal

The Story: Suppose that you are preparing to write a proposal on NGC1365, aiming to investigate the intriguing black hole spin this galaxy with Chandra grating observations (see: https://www.space.com/19980-monster-black-hole-spin-discovery.html ) 

In writing proposals, there are often the same tasks that are required: including finding and analyzing previous observations of the proposal, and creating figures that include, e.g., multiwavelength images and spectrum for the source. 


In [ ]:
# As a hint, we include the code block for Python modules that you will likely need to import:   
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  
import requests, io
from astropy.table import Table

## Use NAVO utility for Registry, cone searches and making images and spectra
import sys
from navo_utils.registry import Registry
from navo_utils.image import Image, ImageColumn
from navo_utils.spectra import Spectra, SpectraColumn
from navo_utils.tap import Tap


## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")


### Step 1: Find out what the previously quoted Chandra 2-10 keV flux of the central source is for NGC 1365.  

Hint: Do a table access protocol (TAP) search in the heasarc, to find potential table with this information

In [ ]:
tap_services=Registry.query(service_type='table',source='heasarc')
url=str(tap_services[0]['access_url'])
# Since there are lots of tables in the HEASARC service, we will narrow the search of Chandra tables by setting contains="chan"
Tap.list_tables(url, contains="chan")

Hint: The Chansngcat ( https://heasarc.gsfc.nasa.gov/W3Browse/chandra/chansngcat.html ) table is likely the best table.  Create a table with ra, dec, exposure time, and flux (and flux errors) from the public.chansngcat catalog for Chandra observations matched within 0.1 degree.

In [ ]:
# list the columns from the query above
Tap.list_columns(url,tablename="public.chansngcat")

In [ ]:
# Get the coordinate for NGC 1365
import astropy.coordinates as coord
pos=coord.SkyCoord.from_name("ngc1365")

In [ ]:
# Show the results from your table query for NGC 1365 and print the ra, dec, exposure time, flux, and flux errors
query="""SELECT ra, dec, exposure, flux, flux_lower, flux_upper FROM public.chansngcat as cat 
    where contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',{},{},0.1))=1 
    and cat.exposure > 0 order by cat.exposure""".format(pos.ra.deg, pos.dec.deg)
results=Tap.query(url,query)
results


### Step 2: Make Images: 

#### Create ultraviolet and X-ray images
Hint: Start by checking what UV image services exist (e.g., GALEX?)

In [ ]:
uv_tables=Registry.query(service_type='image',keyword='galex', waveband='uv')
im_gal = uv_tables[np.isin(uv_tables['short_name'], 
                                    ['GALEX'])]
im_gal['short_name', 'access_url']

Hint: Next create a UV image for the source 

In [ ]:
# Do an image search for NGC 1365 in the UV service found above

im_table=Image.query(uv_tables[0],coords=pos,radius=0.1)
im_table[0]

In [ ]:
# Get the FITS file (which is index 0 for the NUV image or index=2 for the FUV image)
hdu_list=Image.get_fits(im_table[0][0])
# make an image: 
from pylab import figure, cm
from matplotlib.colors import LogNorm
plt.matshow(hdu_list[0].data, origin='lower', cmap=cm.gray_r, norm=LogNorm(vmin=0.005, vmax=0.3))

Hint: Repeat steps for X-ray image. (Note: Ideally, we would find an image in the Chandra 'cxc' catalog) 

In [ ]:
x_tables=Registry.query(service_type='image',waveband='x-ray', source='cxc')
x_tables

In [ ]:
# Do an image search for NGC 1365 in the X-ray CDA service found above
xim_table=Image.query(x_tables[0],coords=pos,radius=0.1)
xim_table[0]

In [ ]:
# Create an image from the first FITS file (index=1) by downloading:
xhdu_list=Image.get_fits(xim_table[0][1])

plt.imshow(xhdu_list[0].data, origin='lower', cmap='cool', norm=LogNorm(vmin=0.1, vmax=500.))
plt.xlim(460, 560)
plt.ylim(460, 560)


### Step 3: Make a spectrum: 

#### Find what Chandra spectral observations exist already for this source. 
Hint: try searching for X-ray spectral data tables using the registry query

In [ ]:
# Use the TAP protocol to list services that contain X-ray spectral data
services=Registry.query(service_type='spectr',waveband='x-ray')
services

In [ ]:
# We want this last listed service and we can identify this by specifying the short_name
chan_tables = services[np.isin(services['short_name'],['Chandra'])]
chan_tables['short_name', 'access_url']

Hint 2: Take a look at what data exist for our candidate, NGC 1365.

In [ ]:
from astropy.coordinates import SkyCoord
target=SkyCoord.from_name("NGC1365")
pos='{},{}'.format(target.ra.deg,target.dec.deg)

spec_tables=Spectra.query(service=chan_tables[0],coords=pos,radius=0.1,verbose=True)

spec_tables[0]['time'].format='.4f'
spec_tables[0].show_in_notebook()



Hint 3: Download the data to make a spectrum. Note: you might end here and use Xspec to plot and model the spectrum. Or ... you can also try to take a quick look at the spectrum. 

In [ ]:
hdu_list=Spectra.get_fits(spec_tables[0][1])
# Or write it to disk
Spectra.get_fits(spec_tables[0][1],filename='downloads/ngc1365_spectrum.fits')

In [ ]:
spec_table=Table(hdu_list[1].data)
spec_table

Extension: Making a "quick look" spectrum. For our purposes, the 1st order of the HEG grating data would be sufficient.

In [ ]:
j=1
for i in range(len(spec_table)):
    matplotlib.rcParams['figure.figsize'] = (8, 3)
    if abs(spec_table['TG_M'][i]) == 1 and (spec_table['TG_PART'][i]) == 1:
        ax=plt.subplot(1,2,j)
        pha = plt.plot( spec_table['CHANNEL'][i],spec_table['COUNTS'][i])
        ax.set_yscale('log')
        if spec_table['TG_PART'][i] == 1:
            instr='HEG'
        ax.set_title("{grating}{order:+d}".format(grating=instr, order=spec_table['TG_M'][i]))
        plt.tight_layout()
        j=j+1


This can then be analyzed in your favorite spectral analysis tool, e.g., [pyXspec](https://heasarc.gsfc.nasa.gov/xanadu/xspec/python/html/index.html).  (For the winter 2018 AAS workshop, we demonstrated this in a [notebook](https://github.com/NASA-NAVO/aas_workshop_2018/blob/master/heasarc/heasarc_Spectral_Access.ipynb) that you can consult for how to use pyXspec, but the pyXspec documentation will have more information.) 

Congratulations! You have completed this notebook exercise.